# Run Neo Inference on EC2

* Launch Deep learning AMI Ubuntu 18.04. Suggest to run at `t3.xlarge`.
* NEO compiled libraray need to run with `GLIBC_2.27`.
* Compile steps please reference folder `6-SageMaker-Neo`

In [ ]:
source activate aws_neuron_mxnet_p36
ldd --version
aws s3 cp s3://beyoung-sagemaker/neo/output/sm-detection-model-ml_c5.tar.gz sm-detection-model-ml_c5.tar.gz
tar xvf sm-detection-model-ml_c5.tar.gz

In [ ]:
mkdir resnet50
mv compiled.params resnet50/model.params
mv compiled_model.json resnet50/model.json
mv compiled.so resnet50/model.so

In [ ]:
pip install dlr

In [ ]:
wget -O dog.jpg https://live.staticflickr.com/4106/5052728523_010830b44e_k_d.jpg

## neo-inference.py

In [ ]:
import dlr
from dlr import DLRModel
import mxnet as mx
import numpy as np
import cv2
from collections import namedtuple

# Load the compiled model
input_shape = {'data': [1, 3, 512, 512]} # A single RGB 512x512 image
output_shape = [1, 80]                 # The probability for each one of the 80 classes
device = 'cpu'                           # CPU
model = DLRModel('resnet50')


filepath = './dog.jpg'
DEFAULT_INPUT_SHAPE = 512
img = cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)
print(img.shape)
img = cv2.resize(img, (DEFAULT_INPUT_SHAPE, DEFAULT_INPUT_SHAPE))
img = np.swapaxes(img, 0, 2)
img = np.swapaxes(img, 1, 2)
img = img[np.newaxis, :]
print(img.shape)

input_data = {'data': img}
# Predict 
out = model.run(input_data)
prob = np.squeeze(out[0])
print(prob.shape)
# Grab result, convert to python list of lists and return
results = [prob[i].tolist() for i in range(10)]
print(results)

```
(aws_neuron_mxnet_p36) ubuntu@ip-172-31-30-187:~$ python neo-inference.py 
(1362, 2048, 3)
(1, 3, 512, 512)
(6132, 6)
[[0.0, 0.5216848850250244, 0.7248552441596985, 0.0, 0.9101921916007996, 0.5743013620376587], [-1.0, -1.0, 0.7113395929336548, 0.0, 0.905834436416626, 0.5425472855567932], [0.0, 0.40109071135520935, 0.35489270091056824, 0.010451018810272217, 0.46446946263313293, 0.5140073299407959], [-1.0, -1.0, 0.32711997628211975, 0.010070711374282837, 0.5042054653167725, 0.629302978515625], [-1.0, -1.0, 0.7220096588134766, 0.024222344160079956, 0.9272550344467163, 0.5419011116027832], [16.0, 0.3286546468734741, 0.12686961889266968, 0.44898146390914917, 0.34717822074890137, 0.7077935338020325], [0.0, 0.32737380266189575, 0.31258323788642883, 0.03207474946975708, 0.508192777633667, 0.6150685548782349], [-1.0, -1.0, 0.3491026759147644, 0.01263803243637085, 0.45894378423690796, 0.5439571142196655], [-1.0, -1.0, 0.35684555768966675, 0.025891810655593872, 0.4664193391799927, 0.4746924936771393], [0.0, 0.30793496966362, 0.7773680090904236, 0.02091062068939209, 0.8835480809211731, 0.509330689907074]]
```

## Viz.py

In [ ]:
filepath = './dog.jpg'
results = [[0.0, 0.5216848850250244, 0.7248552441596985, 0.0, 0.9101921916007996, 0.5743013620376587], [-1.0, -1.0, 0.7113395929336548, 0.0, 0.905834436416626, 0.5425472855567932], [0.0, 0.40109071135520935, 0.35489270091056824, 0.010451018810272217, 0.46446946263313293, 0.5140073299407959], [-1.0, -1.0, 0.32711997628211975, 0.010070711374282837, 0.5042054653167725, 0.629302978515625], [-1.0, -1.0, 0.7220096588134766, 0.024222344160079956, 0.9272550344467163, 0.5419011116027832], [16.0, 0.3286546468734741, 0.12686961889266968, 0.44898146390914917, 0.34717822074890137, 0.7077935338020325], [0.0, 0.32737380266189575, 0.31258323788642883, 0.03207474946975708, 0.508192777633667, 0.6150685548782349], [-1.0, -1.0, 0.3491026759147644, 0.01263803243637085, 0.45894378423690796, 0.5439571142196655], [-1.0, -1.0, 0.35684555768966675, 0.025891810655593872, 0.4664193391799927, 0.4746924936771393], [0.0, 0.30793496966362, 0.7773680090904236, 0.02091062068939209, 0.8835480809211731, 0.509330689907074]]

def visualize_detection(img_file, dets, classes=[], thresh=0.6):
        """
        visualize detections in one image
        Parameters:
        ----------
        img : numpy.array
            image, in bgr format
        dets : numpy.array
            ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
            each row is one object
        classes : tuple or list of str
            class names
        thresh : float
            score threshold
        """
        import random
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg

        img=mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width = img.shape[1]
        colors = dict()
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            ## need to remove class -1
            if klass < 0 or score < thresh:
                continue
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                                 ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id],
                                 linewidth=1)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            plt.gca().text(xmin, ymin - 2,
                            '{:s} {:.3f}'.format(class_name, score),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')
        plt.savefig('result.jpg', dpi=500)
        # plt.show()
        
object_categories = ['person', 'bicycle', 'car',  'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 
                     'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                     'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
                     'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
                     'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                     'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                     'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable',
                     'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
                     'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
                     'toothbrush']

# Setting a threshold 0.20 will only plot detection results that have a confidence score greater than 0.20.
threshold = 0.328
# Visualize the detections.
visualize_detection(filepath, results, object_categories, threshold)


```
(aws_neuron_mxnet_p36) ubuntu@ip-172-31-30-187:~$ python viz.py
Matplotlib is building the font cache using fc-list. This may take a moment.
```

## SCP the result.jpg to local and see the results

```
8c8590168eaf:beyoung-sagemaker beyoung$ scp -i beyoung-sagemaker-west-2-ec2.pem ubuntu@ec2-52-12-68-22.us-west-2.compute.amazonaws.com:~/result.jpg .
result.jpg                                                 100% 2247KB 122.8KB/s   00:18    
```

![](./result.jpg)

## Debug notes of load model

In [25]:
# Load the compiled model
input_shape = {'data': [1, 3, 512, 512]} # A single RGB 512x512 image
output_shape = [1, 80]                 # The probability for each one of the 80 classes
device = 'cpu'                           # CPU
model = DLRModel('resnet50')

###
## DLRError: TVMError: Check failed: lib_handle_ != nullptr: Failed to load dynamic shared library resnet50/model.so /lib64/libm.so.6: version `GLIBC_2.27' not found (required by resnet50/model.so)

2020-05-22 10:00:44,770 ERROR error in DLRModel instantiation TVMError: Check failed: lib_handle_ != nullptr: Failed to load dynamic shared library resnet50/model.so /lib64/libm.so.6: version `GLIBC_2.27' not found (required by resnet50/model.so)
Stack trace:
  File "/root/workplace/centos-6/neo-ai-dlr/3rdparty/tvm/src/runtime/dso_library.cc", line 84
  [bt] (0) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/dlr/libdlr.so(+0x5f79b) [0x7f0c4234579b]
  [bt] (1) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/dlr/libdlr.so(tvm::runtime::Module::LoadFromFile(std::string const&, std::string const&)+0x4d3) [0x7f0c42398613]
  [bt] (2) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/dlr/libdlr.so(dlr::TVMModel::SetupTVMModule(std::vector<std::string, std::allocator<std::string> >)+0xae3) [0x7f0c423386a3]
  [bt] (3) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/dlr/libdlr.so(CreateDLRModel+0x1920) [0x7f0c4231c370]
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/a

DLRError: TVMError: Check failed: lib_handle_ != nullptr: Failed to load dynamic shared library resnet50/model.so /lib64/libm.so.6: version `GLIBC_2.27' not found (required by resnet50/model.so)
Stack trace:
  File "/root/workplace/centos-6/neo-ai-dlr/3rdparty/tvm/src/runtime/dso_library.cc", line 84
  [bt] (0) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/dlr/libdlr.so(+0x5f79b) [0x7f0c4234579b]
  [bt] (1) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/dlr/libdlr.so(tvm::runtime::Module::LoadFromFile(std::string const&, std::string const&)+0x4d3) [0x7f0c42398613]
  [bt] (2) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/dlr/libdlr.so(dlr::TVMModel::SetupTVMModule(std::vector<std::string, std::allocator<std::string> >)+0xae3) [0x7f0c423386a3]
  [bt] (3) /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/dlr/libdlr.so(CreateDLRModel+0x1920) [0x7f0c4231c370]
